In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_pickle('/home/vollrath/Ethiopia/ts_analysis_oldSus/sbae_point_analysis/results_ref/results_Landsat_ndfi_2010-01-01_2013-01-01_2018-01-01_0.25.pickle')

dates = df.head(1).dates.values[0]
dates_float = tf.constant([(date.year + np.round(date.dayofyear/365, 3)) for date in dates])
ts = tf.constant(df.head(1).ts.values[0])
nr_bootstraps = 500

def slope_tf(x, y):
    A = tf.stack([tf.cast(x, tf.float32), tf.ones(len(x))], axis=1)
    y = y[:, tf.newaxis]
    m = tf.linalg.lstsq(A, y)
    return m[0]

In [ ]:
%%timeit
boot_means = []
for _ in range(nr_bootstraps):

    # the fraction of sample we want to include (randon)
    #size = np.abs(np.random.normal(0.5, 0.1))
    size = int(len(ts) * .66)

    # select the random samples
    rand_idx = tf.sort(tf.random.shuffle(tf.constant(tf.range(1, len(ts))))[0:size])
    x = tf.gather(dates_float, indices=rand_idx)
    y = tf.gather(ts, indices=rand_idx)
    
    # calculate the slope on the randomly selected samples
    s = slope(x, y)
    
    # add to list of bootstrap samples
    boot_means.append(s)

# calculate stats adn return
boot_means_np = np.array(boot_means)
print(np.mean(boot_means_np), np.std(boot_means_np), np.max(boot_means_np), np.min(boot_means_np))

In [ ]:
df = pd.read_pickle('/home/vollrath/Ethiopia/ts_analysis_oldSus/sbae_point_analysis/results_ref/results_Landsat_ndfi_2010-01-01_2013-01-01_2018-01-01_0.25.pickle')
dates = df.head(1).dates.values[0]
dates_float = [(date.year + np.round(date.dayofyear/365, 3)) for date in dates]
ts = df.head(1).ts.values[0]
y = np.array(ts)
x = np.array(dates_float)
nr_bootstraps = 500
def slope(x, y):
    A = np.vstack([x, np.ones(len(x))]).T
    
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m

In [ ]:
%%timeit
boot_means = []
for _ in range(nr_bootstraps):


    # the fraction of sample we want to include (randon)
    #size = np.abs(np.random.normal(0.5, 0.1))
    size = 0.66

    # select the random samples
    rand_idx = sorted(np.random.choice(np.arange(y.size), int(y.size * size), replace=False))

    # calculate the slope on the randomly selected samples
    s = slope_tf(x[rand_idx], y[rand_idx])

    # add to list of bootstrap samples
    boot_means.append(s)

# calculate stats adn return
boot_means_np = np.array(boot_means)
print(np.mean(boot_means_np), np.std(boot_means_np), np.max(boot_means_np), np.min(boot_means_np))